# Cats and Dogs: Transferência de Aprendizado

Serão apresentadas as técnicas de *transfer learning* e *fine tunning*. Elas consistem em
utilizar uma rede vencedora da competição anual ImageNet, tanto sua arquitetura como seus
parâmetros treinados em uma outra aplicação envolvendo imagens.

A ideia consiste em utilizar as camadas convolucionais da rede vencedora e trocar apenas as
camadas densas. Num primeiro momento, treina-se apenas as camadas densas (*transfer learning*)
e após este treinamento inicial, continua-se o treinamento com as camadas densas e 
algumas últimas camadas convolucionais da rede (*fine tuning*)

Neste notebook utilizam-se as camadas convolucionais de uma rede ResNet já treinada para gerar as "características" das imagens e treina-se com uma rede neural densa de duas camadas. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display

import os, glob
import PIL.Image

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim.lr_scheduler
import torchvision
from torchvision import transforms
from torchvision import datasets

from lib import pytorch_trainer_v2 as ptt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('GPU available:', device)

GPU available: cuda:0


## Dataset

In [2]:
!wget -nc http://files.fast.ai/data/dogscats.zip -P../data/

File ‘../data/dogscats.zip’ already there; not retrieving.



In [43]:
!unzip -q ../data/dogscats.zip -d ../data
!ls ../data

boston_housing_normalize.npz  dogscats.zip	    MNIST
boston_housing.npz	      Exercicio2.npz	    movielens_norm.npz
dogscats		      Exercicio2_teste.npz  sample_submission.csv


In [3]:
batch_size = 50
#rootdir = '/data/datasets/catsdogs_fewsamples'
rootdir = '../data/dogscats/'
size_final = (224, 224)
transform_noaug = transforms.Compose(
            [transforms.Resize(size=size_final),
             transforms.ToTensor(),
             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

size_large = (256, 256)
transf_comp_train = transforms.Compose([
    transforms.Resize(size=size_large),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(3, translate=None, scale=(0.95, 1.05), shear=None, resample=False, fillcolor=2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=size_final),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


transf_comp_valid = transforms.Compose([torchvision.transforms.Resize(size=size_final),
                                                    torchvision.transforms.ToTensor()])
dataset_train = datasets.ImageFolder(rootdir + '/train/',transform = transform_noaug)  ####
dataset_valid = datasets.ImageFolder(rootdir + '/valid/',transform = transform_noaug)

loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True,  num_workers=4)
loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, num_workers=4)


In [4]:
len(loader_train.dataset), len(loader_valid.dataset)

(23000, 2000)

In [ ]:
if True:
    

## Data Loader

In [23]:
#loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=len(dataset_train), shuffle=True)
#loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
#x_train, y_train = next(iter(loader_train))
#x_valid, y_valid = next(iter(loader_valid))

## Construção da rede neural
O *transfer learning* permite que utilizemos uma rede já treinada (a *ResNet34*, utilizando apenas uma parte convolucional da rede) para gerar um vetor de features. Essa primeira parte do modelo será chamado de *MyResNetConv*.

É possível utilizar esse vetor de features como entrada de uma rede densa simples, que terá como *output* a saída que será comparada com o resultado desejado. Essa parte final do modelo será chamada de *MyResNetDens*.

Essas duas partes serão integradas no modelo *MyResNet*, com o benefício de podermos utilizar a primeira parte para gerar as *features* convolucionais, que serão utilizadas no treinamento exclusivo da segunda parte.

In [5]:
class Funit(nn.Module):
    def forward(self,x):
        return x

class MyResNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.resnet = torchvision.models.resnet34(pretrained=True)    
        for param in self.resnet.parameters():
            param.requires_grad = False
            
        n_feat = self.resnet.fc.in_features
        self.resnet.fc = Funit()
        self.myfc = nn.Linear(n_feat, 2)
        
    def conv(self, x):
        x = self.resnet.forward(x)
        x = x.view(x.size(0), -1)
        return x

    def forward(self, x):
        x = self.conv(x)
        x = self.myfc(x)
        return x


In [6]:
model_fast = MyResNet().to(device)

### Predict das features de uma amostra

In [22]:
x_train, y_train = next(iter(loader_train))
y = model_fast.conv(x_train[:1].to(device))
y.shape

torch.Size([1, 512])

### Predict completo de uma amostra

In [30]:
y = model_fast(x_train[:1].to(device))
y

tensor([[ 0.7571, -0.0491]], device='cuda:0', grad_fn=<ThAddmmBackward>)

## Pré calculando as features

In [7]:
def predict_loader(model, data_loader):
    predictions = []
    labels = []
    with torch.no_grad():
        for k,(X,Y) in enumerate(data_loader):
            Ypred = model(X.to(device))
            Ypred = Ypred.cpu().data
            predictions.append(Ypred)
            labels.append(Y)
            print(k,end='\r')
    return torch.cat(predictions, 0), torch.cat(labels, 0)

In [8]:
model_fast.eval()
f_train, y_train = predict_loader(model_fast.conv,loader_train)
print(f_train.shape)
f_valid, y_valid = predict_loader(model_fast.conv,loader_valid)
print(f_valid.shape)

torch.Size([23000, 512])
torch.Size([2000, 512])


### Predict a partir das features pré calculada de uma amostra

In [35]:
y = model_fast.myfc(f_train[:1].to(device))
y, y_train[0]

(tensor([[ 0.9688, -0.4059]], device='cuda:0', grad_fn=<ThAddmmBackward>),
 tensor(0))

## Treinamento da camada densa utilizando as *features*: mrn_dens

In [9]:
n_epochs = 10
optm = torch.optim.Adam(params=model_fast.myfc.parameters(),lr=0.0001)
schd = torch.optim.lr_scheduler.StepLR(optm, step_size=10, gamma=0.75)
cb_chkpt = ptt.ModelCheckpoint('../../models/transferlearning_features_catsdogs', reset=True, verbose=1)

trainer_tl = ptt.DeepNetTrainer(model         = model_fast.myfc,
                                criterion     = nn.CrossEntropyLoss(),
                                optimizer     = optm,
                                lr_scheduler  = schd,
                                callbacks     = [ptt.AccuracyMetric(), 
                                                ptt.PrintCallback(), 
                                                cb_chkpt],
                                devname       = device)

In [10]:
trainer_tl.fit(n_epochs   = 2,
                   Xin= f_train, Yin= y_train,
                   valid_data=(f_valid, y_valid))


Start training for 2 epochs
  1:   3.4s   T: 0.01102 *  0.96904 *   V: 0.00426 *  0.98750 *  
  2:   3.4s   T: 0.00369 *  0.98913     V: 0.00303 *  0.98900    
Stop training at epoch: 2/2
Best model was saved at epoch 2 with loss 0.00303: ../../models/transferlearning_features_catsdogs


## Fine tunning
É possível ainda refazer o treinamento da camada densa junto com a parte convolucional da rede *ResNet*, utilizando um *learning rate* com valor bem baixo.

In [13]:
for param in model_fast.parameters():
    param.requires_grad = True

## Agora os dados de entrada são os dados originais. Criando os loaders

In [14]:
batch_size = 100
dataset_train = datasets.ImageFolder(rootdir + '/train/',transform = transf_comp_train)
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)

## Preparando para treinar

In [17]:
optm = torch.optim.Adam(params=model_fast.parameters(),  lr=1.e-4)
schd = torch.optim.lr_scheduler.StepLR(optm, step_size=10, gamma=0.75)
cb_chkpt = ptt.ModelCheckpoint('../../models/transferlearning_finetunning_catsdogs', 
                               reset=True, verbose=1)

trainer_finetunning = ptt.DeepNetTrainer(model         = model_fast,
                                         criterion     = nn.CrossEntropyLoss(),
                                         optimizer     = optm,
                                         lr_scheduler  = schd,
                                         callbacks     = [ptt.AccuracyMetric(), 
                                                          ptt.PrintCallback(),
                                                          cb_chkpt],
                                         devname       = device)

In [ ]:
trainer_finetunning.fit_loader(5, loader_train, loader_valid)


Start training for 5 epochs
  1: 283.8s   T: 0.00075 *  0.97339 *   V: 0.00032 *  0.99000 *  


Resultado do treinamento em GPU 1080:

    Start training for 5 epochs
      1:  10.2s   T: 0.10348 0.95750   V: 0.08778 0.96125 best
      2:  10.2s   T: 0.01205 0.99800   V: 0.06722 0.97375 best
      3:  10.2s   T: 0.00294 1.00000   V: 0.09732 0.96500 
      4:  10.2s   T: 0.00121 1.00000   V: 0.06344 0.97625 best
      5:  10.3s   T: 0.00077 1.00000   V: 0.06478 0.98125 
    Stop training at epoch: 5/5
    Best model was saved at epoch 4 with loss 0.06344: ../../models/transferlearning_finetunning_catsdogs

## Exercícios

1. Veja a codificação da rede ResNet (veja o link no github) e explique a inicialização
   da classe do modelo: `super().__init__(BasicBlock, [3, 4, 6, 3])`
2. Observe que a camada densa (`fc`) da ResNet foi suprimida (`del self.fc`). Qual é a
   implicação caso ela não fosse removida? Experimente.
3. Qual é a célula deste notebook que inicializa os pesos de todas as redes?
4. Quantos parâmetros são treinados durante o *transfer learning* da primeira etapa?

## Atividade

1. Utilize a rede ResNet18 e a ResNet50 para fazer o *transfer learning*. 
   Compare os resultados.

## Aprendizados com este notebook
